In [ ]:
import os
import os.path as osp
import subprocess
import numpy as np
import pandas as pd

# Rain Data Exploration

The purpose of this notebook is to explore the rain data from RAWS and HRRR retrieval processes.

* RAWS Rain Data:
    * [RAWS Definitions](https://www.weather.gov/media/wrh/mesowest/MesoWest_Data_Variables_Definitions.pdf)
    * They list hourly rainfall as parameters, but in practice stations only have wide data availability for accumulated precipitation
    * This data is potentially flawed when the rain collecting apparatus is full, and will thus register zero new rainfall when the bucket is full.
* HRRR Rain Data:
    * [HRRR Definitions](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml)
    * Both rate and total ($kg\; m^{-2}s^{-1}$ and $kg\; m^{-2}$, respectively)

In [ ]:
if not osp.exists("co_202306.pkl"):
    print("Retrieving FMDA data")
    subprocess.call("wget -P . https://demo.openwfm.org/web/data/fmda/dicts/co_202306.pkl", shell=True)
    assert osp.exists("co_202306.pkl")    
    print("Downloaded https://demo.openwfm.org/web/data/fmda/dicts/co_202306.pkl as co_202306.pkl")